# Scheduling Optimization

## Introduction

This exercise has for purpose to solve scheduling issues with assigning auditors to specific projects throughout the course of a year. There is a lot of information that goes into building such a model with solutions that may never be optimal or make everyone involved happy, but it is a start in helping organizations better use their resources on a more granular level.

## Setting up the base model

The first step in the process is simply to import the package that we need, which in this case is the cp_model from OR-Tools and build the model. We can then set up the base settings where: <br> <br>

workers: number of auditors <br>
shifts: number of auditors needed per audit <br>
days: number of audit <br>
maxshiftsperday: 1 regardless, since an auditor can't have multiple roles on the same project <br>
maxdifference: 1 to ensure an auditor is not assigned to more audits than another auditor <br> <br>

We then input all the schedule possibilities into the model use nested for loops.

In [1]:
# importing package
from ortools.sat.python import cp_model

# creating model
model = cp_model.CpModel()

# storing indices for worker, shifts and day combination
shiftoptions = {}

In [2]:
# setting up base information for schedules
workers = 5
shifts = 3
days = 5
maxshiftsperday = 1
maxdifference = 1

In [3]:
# creating each possible combination of worker, shift and day
for x in range(workers):
    for y in range(days):
        for z in range(shifts):
            shiftoptions[(x,y,z)] = model.NewBoolVar("shift with id" + str(x) + " " + str(y) + " " + str(z))

## Adding constraints

This is where organization specific needs are implemented into the model. For the current case that is related to auditor scheduling, we have currently added the following constraints <br> <br>

Each audit slot can only be assigned to 1 worker <br>
Auditor can only work on 1 audit at a time <br>
Same number of audits for every for every auditor <br>


In [4]:
# one auditor per audit slot
for y in range(days):
    for z in range(shifts):
        model.Add(sum(shiftoptions[(x, y, z)] for x in range(workers)) == 1)

In [5]:
# auditors can only work one audit at a time
for x in range(workers):
    for y in range(days):
        model.Add(sum(shiftoptions[(x,y,z)] for z in range(shifts)) <= 1)

In [6]:
# auditors should have the same (or almost) number of audit assignments
minshiftsperworker = (shifts * days) // workers
maxshiftsperworker = minshiftsperworker + maxdifference
for x in range(workers):
    shiftsassigned = 0
    for y in range(days):
        for z in range(shifts):
            shiftsassigned += shiftoptions[(x,y,z)]
    model.Add(minshiftsperworker <= shiftsassigned)
    model.Add(shiftsassigned <= maxshiftsperworker)

Given that this is the first draft of such an optimization problem, there are still lots of constraints that will need to get implemented to ensure that it can be applied to the real world. Examples of such constraints could be: <br>
<br>
Overlaps between audits<br>
Auditor turnaround (people joining or leaving)<br>
Employee hierarchy (ap1 vs dx)<br>
Familiarity of an auditor with an audit<br>
Auditor preference for specific audits

## Solving problem and printing solution

This is the easy part where we are simply solving the problem that we have previously set up and printing its solution in a manner that a human being can understand.

In [7]:
# building a class to print the solution
class SolutionPrinterClass(cp_model.CpSolverSolutionCallback):
    def __init__(self, shiftoptions, workers, days, shifts, sols):
        val = cp_model.CpSolverSolutionCallback.__init__(self)
        self._shiftoptions = shiftoptions
        self._workers = workers
        self._days = days
        self._shifts = shifts
        self._solutions = set(sols)
        self._solution_count = 0
    def on_solution_callback(self):
        if self._solution_count in self._solutions:
            print("Solution #" + str(self._solution_count + 1))
            for y in range(self._days):
                print("Month " + str(y))
                for x in range(self._workers):
                    is_working = False
                    for z in range(self._shifts):
                        if self.Value(self._shiftoptions[(x,y,z)]):
                            is_working = True
                            print("Auditor " + str(x) + " for audit #" + str(z))
                    #if not is_working:
                        #print('  Auditor {} does not work'.format(x))
            print()
        self._solution_count += 1
    def solution_count(self):
        return self._solution_count

In [ ]:
# solving the scheduling problem
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solutionrange = range(2)
solution_printer = SolutionPrinterClass(shiftoptions, workers,days , shifts, solutionrange)
solver.SearchForAllSolutions(model, solution_printer)

Solution #1
Month 0
Auditor 2 for audit #1
Auditor 3 for audit #2
Auditor 4 for audit #0
Month 1
Auditor 1 for audit #0
Auditor 3 for audit #1
Auditor 4 for audit #2
Month 2
Auditor 0 for audit #2
Auditor 3 for audit #1
Auditor 4 for audit #0
Month 3
Auditor 0 for audit #2
Auditor 1 for audit #0
Auditor 2 for audit #1
Month 4
Auditor 0 for audit #1
Auditor 1 for audit #2
Auditor 2 for audit #0

Solution #2
Month 0
Auditor 2 for audit #1
Auditor 3 for audit #2
Auditor 4 for audit #0
Month 1
Auditor 0 for audit #0
Auditor 3 for audit #1
Auditor 4 for audit #2
Month 2
Auditor 1 for audit #2
Auditor 3 for audit #1
Auditor 4 for audit #0
Month 3
Auditor 0 for audit #2
Auditor 1 for audit #0
Auditor 2 for audit #1
Month 4
Auditor 0 for audit #1
Auditor 1 for audit #2
Auditor 2 for audit #0



There are multiple solutions in this case that would solve all of our constraints, though in a real scenario there may not be any feasible solutions. It is a caveat that needs to be taken into consideration before any full scale useage.